In [1]:
import pandas as pd
import sys
sys.path.append('../pycore/')
from simulate import RBA_result
from utils import extract_details_from_rxnid

In [2]:
df_prot = pd.read_excel('../scRBA_build_GAMS_model/input/PROTEIN_stoich_curation.xlsx')
df_prot.index = df_prot.id.to_list()
protein_mw = {i:df_prot.loc[i, 'MW (g/mmol)'] for i in df_prot.index}
protein_mw['PROTDUMMY'] = 44.026303

df_enz = pd.read_excel('../scRBA_build_GAMS_model/input/ENZYME_stoich_curation_2021-10-01.xlsx')
df_enz.index = df_enz.id.to_list()
enzyme_mw = {'ENZLOAD-'+i[4:]:df_enz.loc[i, 'MW (g/mmol)'] for i in df_enz.index}

res = RBA_result(protein_mw=protein_mw, enzyme_mw=enzyme_mw)
res.load_and_calculate('./min_flux_sum.flux.txt')

In [3]:
print('EX_glc__D_e', res.metabolic_flux['EX_glc__D_e'])
print('EX_o2_e', res.metabolic_flux['EX_o2_e'])
for k,v in res.metabolic_flux.items():
    if k[:3] == 'EX_' and v > 1e-6:
        print(k,v)

EX_glc__D_e -13.2
EX_o2_e -3.661257223231814
EX_co2_e 24.06212277021086
EX_etoh_e 19.580000000000013
EX_gcald_e 2.5470073272547e-05
EX_h_e 0.8666230236175908
EX_h2o_e 12.73197283747805


In [4]:
#### Map rxn to enz
rxndict = {k:[] for k,v in res.metabolic_flux.items() if abs(v) > 0}

enzdict = dict()
for k,v in res.raw_flux.items():
    if k.split('-')[0] == 'ENZSYN' and v > 0:
        _,enz = k.split('-', maxsplit=1)
        enzdict[enz] = []
        
for i in df_enz.index:
    rxn = df_enz.rxn_src[i]
    enz = df_enz.enz[i]
    if rxn in rxndict.keys() and enz in ['SPONT', 'UNKNOWN']:
        rxndict[rxn].append('zeroCost')
        
    if rxn in rxndict.keys() and enz in enzdict.keys():
        rxndict[rxn].append(enz)
        enzdict[enz].append(rxn)
        
rxndict = {k:set(v) for k,v in rxndict.items()}
rxndict = {k:v for k,v in rxndict.items() if v != {'zeroCost'}}
enzdict = {k:set(v) for k,v in enzdict.items()}

In [5]:
# Find enzymes whose together carry a total load of reactions
x = {k:v for k,v in rxndict.items() if len(v) > 1.5}
enz_share_rxn_load = set().union(*[v for v in x.values()])

# Find enzymes whose individually carry loads of multiple reactions
x = {k:v for k,v in enzdict.items() if len(v) > 1.5}
enz_multiload = set(x.keys())

# Set 1: Enzyme-reaction one-to-one load mapping
set1 = set(enzdict.keys()) - enz_share_rxn_load - enz_multiload
set1 = set([i for i in set1 if len(enzdict[i]) > 0.5])

# Set 2: Enzyme-reaction one-to-many
set2 = enz_multiload - enz_share_rxn_load

# Set 3: Enzyme-reaction many-to-one
set3 = enz_share_rxn_load - enz_multiload

# Set 4: Enzyme-reaction many-to-many
set4 = enz_share_rxn_load & enz_multiload

mu = 0.407
kapp = dict()
for enz in set1:
    rxn = [i for i in enzdict[enz]][0]
    rval = res.metabolic_flux[rxn]
    if rval > 0:
        rdir = 'FWD'
    elif rval < 0:
        rdir = 'REV'
    else:
        print('rval == 0, check enzyme ' + enz + ' and reaction ' + rxn)
    
    rid = 'RXN-' + rxn + '_' + rdir + '-' + enz
    
    enzval = res.raw_flux['ENZSYN-' + enz]
    kapp[rid] = abs(rval) / enzval / mu / 3600
    
for enz in set2:
    rids = []; rvalsum = 0;
    for rxn in enzdict[enz]:
        rval = res.metabolic_flux[rxn]
        if rval > 0:
            rdir = 'FWD'
        elif rval < 0:
            rdir = 'REV'
        else:
            print('rval == 0, check enzyme ' + enz + ' and reaction ' + rxn)

        rids.append('RXN-' + rxn + '_' + rdir + '-' + enz)
        rvalsum += abs(rval)
        
    enzval = res.raw_flux['ENZSYN-' + enz]
    
    for rid in rids:
        kapp[rid] = rvalsum / enzval / mu / 3600
        
rxns = set().union(*[enzdict[enz] for enz in set3])
for rxn in rxns:
    rval = res.metabolic_flux[rxn]
    if rval > 0:
        rdir = 'FWD'
    elif rval < 0:
        rdir = 'REV'
    else:
        print('rval == 0, check enzyme ' + enz + ' and reaction ' + rxn)
        
    rids = []; enzvalsum = 0;
    for enz in rxndict[rxn]:
        rids.append('RXN-' + rxn + '_' + rdir + '-' + enz)
        enzvalsum += res.raw_flux['ENZSYN-' + enz]
        
    for rid in rids:
        kapp[rid] = rval / enzvalsum / mu / 3600
        
with open('./kapp_ambiguousLoad_case_resolve.txt') as f:
    text = f.read().split('\n')[1:]
text = [i for i in text if i != '']
mapper = dict()
for line in text:
    k,v = line.split('\t')
    if v != 'noRxnFlux_resolveByManualCheck':
        mapper[k] = v
        
for enztext,rxntext in mapper.items():
    enzs = enztext.split(',')
    rxns = rxntext.split(',')
    
    enzvalsum = sum([res.raw_flux['ENZSYN-'+enz] for enz in enzs])
    rxnvalsum = sum([abs(res.metabolic_flux[rxn]) for rxn in rxns])
    
    rids = []
    for rxn in rxns:
        rval = res.metabolic_flux[rxn]
        if rval > 0:
            rdir = 'FWD'
        elif rval < 0:
            rdir = 'REV'
        else:
            print('rval == 0, check enzyme ' + enz + ' and reaction ' + rxn)
            
        for enz in enzs:
            rids.append('RXN-' + rxn + '_' + rdir + '-' + enz)
            
    for rid in rids:
        kapp[rid] = rvalsum / enzvalsum / mu / 3600

#### Write calculated kapp (1/s)

In [9]:
texts = ['rxnid\tkapp (1/s)']
for k,v in kapp.items():
    texts.append(k + '\t' + str(v))
    
with open('./kapps_in_vivo.txt', 'w') as f:
    f.write('\n'.join(texts))

In [58]:
enz = 'YPR074C'
print(enz + ':', enzdict[enz])
for rxn in enzdict[enz]:
    print(rxn + ':', rxndict[rxn])

x = set().union(*[rxndict[rxn] for rxn in enzdict[enz]])
for enz2 in x:
    if enz2 != enz:
        print(enz2 + ':', enzdict[enz])

YPR074C: {'TKT1_c', 'TKT2_c'}
TKT1_c: {'YPR074C', 'YBR117C'}
TKT2_c: {'YPR074C', 'YBR117C'}
YBR117C: {'TKT1_c', 'TKT2_c'}
